In [1]:
import os
import sys
import pandas as pd
import numpy as np

from sklearn.metrics import mean_absolute_percentage_error, root_mean_squared_error

# Add the parent directory to the Python path to load funtions from file ML_funtions
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
sys.path.append(parent_directory)

# Import helperfunctions
from ML_functions import fun_load_data, fun_preprocessing, fun_benchmark_evaluation

# **A) TSP benchmarks**

In [2]:
# Assign string "TSP" or "CVRP" to the following variable to define the optimization problem
optimization_problem = "TSP_benchmarks"

# Load data
data, start_script = fun_load_data(optimization_problem)

# Do the train test split during the preprocessing
X_train, X_test, y_train, y_test, train_data = fun_preprocessing(data, train_size=0.8, keep_SHAPO=True)

### **Evaluation of the *SHAPO* benchmark defined in Levinger et al., 2021**

In [3]:
# Display the Data Frame with all predictions of the two proxies SHAPO and Φ BLEND
display(train_data)

# Check if all instances of SHAPO are scaled
a = []

print("Check whether all instances are scaled correctly:")
for id in range(1, max(train_data.index.get_level_values(level="Instance ID")) + 1):
    sum_shapo = train_data.xs(id, level="Instance ID")["SHAPO"].sum()
    total_cost = train_data.xs(id, level="Instance ID")["Total Cost"].iloc[0]
    sum_shapo = np.round(sum_shapo, 2)
    total_cost = np.round(total_cost, 2)
    a.append(sum_shapo == total_cost)
    if (sum_shapo != total_cost):
        print(" - Instance {}: Sum of predictions: {}, Total Cost: {}".format(id, sum_shapo, total_cost))

print("Number of instances incorrect scaled (Sum of predictins is not equal to the total cost):", np.sum([i == False for i in a]))

,,Number Customers,X,Y,X Depot,Y Depot,Depot Distance,Total Cost,Shapley Value,SHAPO,Φ DEPOT,Φ MOAT,Φ BLEND
Index,Instance ID,,,,,,,,,,,,
0,1,6,11.757432,50.848731,2.380844,66.016752,17.832253,227.291186,6.805996,6.847093,12.131437,30.438918,14.026705
1,1,6,83.228495,41.537025,2.380844,66.016752,84.472480,227.291186,73.361446,72.126416,57.467361,53.908965,69.969918
2,1,6,33.032921,29.876631,2.380844,66.016752,47.388376,227.291186,21.568006,21.751330,32.238724,31.852306,33.359091
3,1,6,42.131509,30.755973,2.380844,66.016752,53.136032,227.291186,25.980268,26.403603,36.148904,33.513918,37.963247
4,1,6,54.103013,58.267699,2.380844,66.016752,52.299433,227.291186,30.191952,30.346193,35.579758,31.791732,36.391426
...,...,...,...,...,...,...,...,...,...,...,...,...,...
89995,9000,14,35.411268,13.512220,83.308855,83.076767,84.459488,370.421950,32.946966,33.551630,38.970270,26.559562,36.319218
89996,9000,14,94.027698,4.059342,83.308855,83.076767,79.741125,370.421950,70.014213,69.654750,36.793181,32.775583,38.105393
89997,9000,14,15.703090,62.406463,83.308855,83.076767,70.695127,370.421950,37.349798,42.540654,32.619286,28.146275,31.451366


Check whether all instances are scaled correctly:
 - Instance 4575: Sum of predictions: 342.24, Total Cost: 341.96
 - Instance 7130: Sum of predictions: 383.16, Total Cost: 382.87
 - Instance 7618: Sum of predictions: 344.32, Total Cost: 344.15
Number of instances incorrect scaled (Sum of predictins is not equal to the total cost): 3


In [4]:
# Create a dict to store the results of the benchmarks
results_dict = {}

# Evaluate the SHAPO benchmark
results_dict = fun_benchmark_evaluation(X_train, X_test, y_train, y_test, benchmark_str="SHAPO", results_dict=results_dict)

,Train set,Test set
MAPE,3.82,3.77
RMSE,1.62,1.60


Number Customers,6,7,8,9,10,11,12,13,14,Mean
MAPE,1.31,1.60,2.26,2.90,3.46,4.20,4.51,5.08,5.37,3.77
RMSE,0.91,0.98,1.14,1.36,1.53,1.77,1.74,1.88,1.92,1.60


### **Evaluation of the *Depot Distance* benchmark defined in Aziz et al., 2016**

In [5]:
# Evaluate the "Depot Distance" benchmark
results_dict = fun_benchmark_evaluation(X_train, X_test, y_train, y_test, benchmark_str="Φ DEPOT", results_dict=results_dict)

,Train set,Test set
MAPE,29.01,28.94
RMSE,9.80,9.81


Number Customers,6,7,8,9,10,11,12,13,14,Mean
MAPE,28.34,27.76,27.90,28.55,29.06,30.37,29.35,29.11,28.93,28.94
RMSE,11.65,11.28,10.57,10.14,9.76,9.80,9.21,9.17,8.52,9.81


### **Evaluation of the *Blended Proxy* benchmark defined in Aziz et al., 2016**

In [6]:
# Evaluate the "Depot Distance" benchmark
results_dict = fun_benchmark_evaluation(X_train, X_test, y_train, y_test, benchmark_str="Φ BLEND", results_dict=results_dict)

,Train set,Test set
MAPE,27.86,28.04
RMSE,9.09,9.09


Number Customers,6,7,8,9,10,11,12,13,14,Mean
MAPE,38.11,32.83,30.53,28.62,27.10,26.24,25.60,24.89,26.62,28.04
RMSE,11.47,10.28,9.38,8.91,8.71,8.85,8.53,8.74,8.46,9.09


### **Store the results**

In [7]:
# Get a list with the benchmark names
benchmarks = list(results_dict.keys())

# Get scores of all benchmarks
MAPE_train_scores = [value[0].loc["MAPE", "Train set"] for value in results_dict.values()]
MAPE_test_scores = [value[0].loc["MAPE", "Test set"] for value in results_dict.values()]
RMSE_train_scores = [value[0].loc["RMSE", "Train set"] for value in results_dict.values()]
RMSE_test_scores = [value[0].loc["RMSE", "Test set"] for value in results_dict.values()]
MAPE_cat_scores = [value[1].loc["MAPE"] for value in results_dict.values()]
RMSE_cat_scores = [value[1].loc["RMSE"] for value in results_dict.values()]

# Show train and test scores for each benchmark a Data Frame
MAPE_df = pd.DataFrame(data=[MAPE_train_scores, MAPE_test_scores], columns=benchmarks, index=["Train set", "Test set"]).sort_values(by="Test set", axis=1)
MAPE_df.columns.name = "MAPE scores"
RMSE_df = pd.DataFrame(data=[RMSE_train_scores, RMSE_test_scores], columns=benchmarks, index=["Train set", "Test set"]).sort_values(by="Test set", axis=1)
RMSE_df.columns.name = "RMSE scores"
display(MAPE_df, RMSE_df)

# Show scores per instance size for each benchmark
MAPE_cat_scores_df = pd.DataFrame(data=MAPE_cat_scores, index=benchmarks).sort_values(by="Mean")
MAPE_cat_scores_df.columns.name = "MAPE scores per instance size"
RMSE_cat_scores_df = pd.DataFrame(data=RMSE_cat_scores, index=benchmarks).sort_values(by="Mean")
RMSE_cat_scores_df.columns.name = "RMSE scores per instance size"
display(MAPE_cat_scores_df, RMSE_cat_scores_df)

# Save data frames with results into an excel file
file_path = str(f"..\\04_test_results/{optimization_problem}.xlsx")

# Use ExcelWriter to write multiple DataFrames to the same file
with pd.ExcelWriter(file_path) as writer:
    MAPE_df.to_excel(writer, sheet_name="MAPE_scores")
    RMSE_df.to_excel(writer, sheet_name="RMSE_scores")
    MAPE_cat_scores_df.to_excel(writer, sheet_name="MAPE_cat_scores")
    RMSE_cat_scores_df.to_excel(writer, sheet_name="RMSE_cat_scores")
print("File saved succesfully.")

MAPE scores,SHAPO,Φ BLEND,Φ DEPOT
Train set,3.82,27.86,29.01
Test set,3.77,28.04,28.94


RMSE scores,SHAPO,Φ BLEND,Φ DEPOT
Train set,1.62,9.09,9.80
Test set,1.60,9.09,9.81


MAPE scores per instance size,6,7,8,9,10,11,12,13,14,Mean
SHAPO,1.31,1.60,2.26,2.90,3.46,4.20,4.51,5.08,5.37,3.77
Φ BLEND,38.11,32.83,30.53,28.62,27.10,26.24,25.60,24.89,26.62,28.04
Φ DEPOT,28.34,27.76,27.90,28.55,29.06,30.37,29.35,29.11,28.93,28.94


RMSE scores per instance size,6,7,8,9,10,11,12,13,14,Mean
SHAPO,0.91,0.98,1.14,1.36,1.53,1.77,1.74,1.88,1.92,1.60
Φ BLEND,11.47,10.28,9.38,8.91,8.71,8.85,8.53,8.74,8.46,9.09
Φ DEPOT,11.65,11.28,10.57,10.14,9.76,9.80,9.21,9.17,8.52,9.81


File saved succesfully.


# **B) CVRP benchmark**

In [8]:
# Assign string "TSP" or "CVRP" to the following variable to define the optimization problem
optimization_problem = "CVRP_benchmark"

# Load data
data, start_script = fun_load_data(optimization_problem)

# Compute benchmark "Depot Distance"
# Add the fraction of the depot distance in the total depot distance of the instance for each customer and multiply it with the total cost
data["Φ DEPOT"] = data.groupby("Instance ID").apply(
    lambda group: (group["Depot Distance"] / group["Depot Distance"].sum()) * group["Total Cost"]
    ).reset_index(drop=True)
data = data[["Instance ID", "Number Customers", "X", "Y", "X Depot", "Y Depot", 
                     "Depot Distance", "Total Cost", "Shapley Value", "Φ DEPOT"]]
display(data.head(10))

# Do the train test split during the preprocessing
X_train, X_test, y_train, y_test, train_data = fun_preprocessing(data, train_size=0.8)

,Instance ID,Number Customers,X,Y,X Depot,Y Depot,Depot Distance,Total Cost,Shapley Value,Φ DEPOT
0,1,5,86.027966,95.446677,29.551338,5.437034,106.260742,290.56169,85.337134,85.193945
1,1,5,92.299886,89.092199,29.551338,5.437034,104.573261,290.56169,72.157370,83.841016
2,1,5,61.404822,7.793238,29.551338,5.437034,31.940509,290.56169,30.779912,25.608121
3,1,5,57.101808,4.267994,29.551338,5.437034,27.575262,290.56169,28.850226,22.108309
4,1,5,67.005710,89.535835,29.551338,5.437034,92.062143,290.56169,73.437048,73.810299
5,2,5,43.183837,93.762299,24.420255,33.242149,63.362138,246.17461,50.910365,57.748040
6,2,5,79.986244,64.675321,24.420255,33.242149,63.840610,246.17461,48.553048,58.184117
7,2,5,83.763369,55.743989,24.420255,33.242149,63.466038,246.17461,53.444287,57.842734
8,2,5,3.806485,84.479076,24.420255,33.242149,55.228165,246.17461,54.846325,50.334764
9,2,5,2.456177,23.058554,24.420255,33.242149,24.210046,246.17461,38.420585,22.064955


### **Evaluation of the *Depot Distance* defined in Aziz et al., 2016**

In [9]:
# Evaluate the "Depot Distance" benchmark
results_dict = fun_benchmark_evaluation(X_train, X_test, y_train, y_test, benchmark_str="Φ DEPOT", results_dict=results_dict)

,Train set,Test set
MAPE,23.08,23.61
RMSE,11.08,11.16


Number Customers,5,6,7,8,9,10,11,12,Mean
MAPE,21.40,22.31,22.07,23.23,23.03,23.85,25.12,25.20,23.61
RMSE,10.66,11.17,11.06,11.35,10.88,11.26,11.54,11.04,11.16


### **Store the results**

In [10]:
# Get a list with the benchmark names
benchmarks = list(results_dict.keys())

# Get scores of the benchmarks
scores_df = results_dict["Φ DEPOT"][0]
cat_scores_df = results_dict["Φ DEPOT"][1]

# Save data frames with results into an excel file
file_path = str(f"..\\04_test_results/{optimization_problem}_depot.xlsx")

# Use ExcelWriter to write multiple DataFrames to the same file
with pd.ExcelWriter(file_path) as writer:
    scores_df.to_excel(writer, sheet_name="scores")
    cat_scores_df.to_excel(writer, sheet_name="cat_scores")
print("File saved succesfully.")

File saved succesfully.
